In [1]:
import tweepy
from pprint import pprint
import jsonpickle
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
auth = tweepy.OAuthHandler('iEk2lB2S9h6qdi9bG2F3OOfVX','LHOn4yNXL5ggl193syNeYaMs2jHGlw7zwDVLFc2pjVWW9XD5LO')
auth.set_access_token('1450808706-r0tIB7jGbOeHbz5EiXkUed2HdnTyNqZFGSZINTL','UCkmKWNSK6EYvk5MTT75GJERl7MPwGOMEXAsjqFnhmlBw')

api = tweepy.API(auth)
if (not api):
    print("Authentication failed :(")
else:
    print("Authentication successfull!!! :D")

Authentication successfull!!! :D


In [3]:
df = pd.read_csv(
    'corpus.txt', 
    delimiter='\t', 
    names=['event_mention_id', 'tweet_id', 'event_instance_id', 'event_mention_trigger', 'timestamp', 'text']
)

In [4]:
df1 = {
    'tweet_id':list(),
    'text':list()
}
tweets = []
dataset = []
tweet_ids = df['tweet_id'].values.tolist()

In [ ]:
for i in tqdm(range(0, len(tweet_ids), 100)):
    statuses = api.statuses_lookup(tweet_ids[i: i + 100])
    dataset.append(statuses)
    
    for idx, status in enumerate(statuses):
        tweet = df.iloc[[idx]]
        df1['tweet_id'].append(status._json['id'])
        df1['text'].append(status._json['text'])
    df_prime = pd.DataFrame(df1)
    df_prime.to_csv('fetched_tweets.csv')
    time.sleep((15 * 60) + 2)

In [4]:
df1 = pd.read_csv('fetched_tweets.csv')

In [11]:
df1

,tweet_id,text
0,784777229632962560,.@MarthaRaddatz @andersoncooper Ask @realDonal...
1,784762671232983040,Federal Judge Orders Hearing in Donald Trump R...
2,784586044897394688,@sddickerson @FightandDebate @HillaryClinton @...
3,784751011063816192,What leaked emails reveal about Hillary Clinto...
4,784691959445282816,@DrCrane72 @realDonaldTrump he bragged about h...
...,...,...
2047,790333734553849856,Black Trump Supporters Explain Why They are Vo...
2048,793631980000862208,Judge Curiel allows Trump University case to c...
2049,791383816300560385,@ComedyCentral's @TheDailyShow with @Trevornoa...
2050,793629375371546624,Trump Threatened To Make The 13-Year-Old Girl ...


In [12]:
def transform(line):
    line = line.lower()
    line = line.replace("'''", "'")
    line = line.replace("isn't", "is not")
    line = line.replace("wasn't", "was not")
    line = line.replace("didn't", "did not")
    line = line.replace("don't", "do not")
    line = line.replace("dont", "do not")

    line = line.replace("doesn't", "does not")
    line = line.replace("doesnt", "does not")

    line = line.replace("won't", "will not")
    line = line.replace("wont", "will not")

    line = line.replace("wouldn't", "would not")
    line = line.replace("can't", "can not")
    line = line.replace("cannot", "can not")

    line = line.replace("hasn't", "has not")
    line = line.replace("*questioned", "questioned")
    line = line.replace("targets", "target")
    line = line.replace("=", "")
        
    return line


In [13]:
import tokenizer as tk
import re

In [14]:
df1 = pd.DataFrame(df1)
def cleanTweet(tweet):
    s = " ".join(tk.tokenize(tweet[:-1]))
    x = re.sub(r'[^\x00-\xf3]', '', s)
    return x


In [15]:
df_main = {
    'event_mention_id': list(), 
    'tweet_id': list(), 
    'event_instance_id': list(), 
    'event_mention_trigger': list(), 
    'timestamp': list(), 
    'text':list()
}

In [16]:
for tweet_id in df1['tweet_id'].values:
    tweet = df[tweet_id == df['tweet_id']]
    for key in df_main.keys():
        if key != 'text':
            df_main[key].append(tweet[key].values[0])
    df_main['text'].append(df1[df1['tweet_id'] == tweet_id]['text'].values[0])

In [17]:
df_main = pd.DataFrame(df_main)

In [18]:
df_main['text'] = df_main['text'].apply(transform)
df_main['text'] = df_main['text'].apply(cleanTweet)

In [19]:
df_main

,event_mention_id,tweet_id,event_instance_id,event_mention_trigger,timestamp,text
0,98,784777229632962560,48,boasting,1475940446657,. @martharaddatz @andersoncooper ask @realdona...
1,84,784762671232983040,246,orders,1475936975664,federal judge orders hearing in donald trump r...
2,14,784586044897394688,201,paid,1475894864664,@sddickerson @fightanddebate @hillaryclinton @...
3,76,784751011063816192,118,reveal,1475934195663,what leaked emails reveal about hillary clinto...
4,61,784691959445282816,48,bragged,1475920116660,@drcrane72 @realdonaldtrump he bragged about h...
...,...,...,...,...,...,...
2047,3149,790333734553849856,1005,explain,1477265220665,black trump supporters explain why they are vo...
2048,330,793631980000862208,1440,allows,1478051583658,judge curiel allows trump university case to c...
2049,3158,791383816300560385,999,pledge to sue,1477515579663,@comedycentral 's @thedailyshow with @trevorno...
2050,327,793629375371546624,1245,threatened to make disappear,1478050962666,trump threatened to make the 13-year-old girl ...


In [10]:
df_main = pd.read_csv(
    'generated_dataset.txt', 
    sep='\t', 
    names=[
        'event_mention_id', 
        'tweet_id', 
        'event_instance_id', 
        'event_mention_trigger', 
        'timestamp', 
        'text'
    ]
)

In [12]:
df_main

,event_mention_id,tweet_id,event_instance_id,event_mention_trigger,timestamp,text
0,0,784576649627103233,48,bragging,1475892624657,drools over a 12-year old and rapes a 13-year ...
1,1,784576649627103233,1514,rapes,1475892624657,drools over a 12-year old and rapes a 13-year ...
2,5,784579912812400641,17,is under attack,1475893402661,@thehill @wsj @washtimes @foxnews trump is und...
3,6,784579912812400641,1495,helping,1475893402661,@thehill @wsj @washtimes @foxnews trump is und...
4,8,784581821233242114,53,attack,1475893857664,republicans attack donald trump following emer...
...,...,...,...,...,...,...
1768,2807,791598375917486081,1097,does not endorse,1477566734662,boingboing : the yale record does not endorse ...
1769,2810,791626364486946816,1116,presses,1477573407657,clinton supporter presses trump surrogate on s...
1770,2811,791628209997680641,1116,presses,1477573847661,clinton supporter presses trump surrogate on s...
1771,2812,791638100149669888,1097,does not endorse,1477576205657,"quite funny semantics here . "" the yale record..."


In [20]:
with open('FinalDataset.csv', 'w') as f:
    for idx in df_main.index:
        row = df_main.iloc[[idx]]
        print(f"{row['event_mention_id'].values[0]}\t{row['tweet_id'].values[0]}\t{row['event_instance_id'].values[0]}\t{row['event_mention_trigger'].values[0]}\t{row['timestamp'].values[0]}\t{row['text'].values[0]}", file=f)


# Word2Vec

In [21]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [22]:
class Tweet_Reader:
    def __init__(self, df):
        self.dataframe = df

    def __iter__(self):
        for idx in self.dataframe.index:
            temp = self.dataframe['text'][idx]
            temp = tk.tokenize(temp.lower())
            yield temp

In [23]:
path = get_tmpfile("word2vec.model")
tweets = Tweet_Reader (df_main) # a memory-friendly iterator

model = Word2Vec(tweets, size=100, window=5, min_count=1, workers=4, iter=30)
model
model.save("word2vec.model")

In [24]:
from torchtext.data import Field
from tqdm.notebook import tqdm
import torch
from torchtext.vocab import Vocab

## Construct Vocab

In [25]:
v = {}
for word, vocab_obj in model.wv.vocab.items():
    v[word] = vocab_obj.count
v['<unk>'] = 0
v['<pad>'] = 0

In [26]:
vc = Vocab(v, specials=['<pad>'])

In [27]:
word2vec_vectors = []
for token, idx in tqdm(vc.stoi.items()):
    if token in model.wv.vocab.keys():
        word2vec_vectors.append(torch.FloatTensor(model[token]))
    else:
        word2vec_vectors.append(torch.zeros(100))

/home/rizwan/anaconda3/envs/ire/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/home/rizwan/anaconda3/envs/ire/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  after removing the cwd from sys.path.


In [28]:
vc.set_vectors(vc.stoi, word2vec_vectors, 100)

In [29]:
pre_trained_emb = torch.FloatTensor(vc.vectors)

In [30]:
embedding = torch.nn.Embedding.from_pretrained(pre_trained_emb)

In [31]:
embedding(torch.Tensor([vc.stoi['<pad>']]).long())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [32]:
import pickle

In [33]:
with open("vocab.pkl", 'wb') as f:
    pickle.dump(vc, f)